In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [17]:
lemmatizer = WordNetLemmatizer()

In [3]:
conversation = json.loads(open("conversation.json").read())

In [23]:
words = []
classes = []
documents = []
ignore_letters = [".",",","?","!"]

In [5]:
for con in conversation["conversation"]:
    for pat in con["patterns"]:
        word_list = nltk.word_tokenize(pat)
        words.append(word_list)
        documents.append((word_list, con["tag"]))
        if con['tag'] not in classes:
            classes.append(con['tag'])

In [10]:
print(documents)

[(['Hello'], 'greetings'), (['Hi'], 'greetings'), (['Hey'], 'greetings'), (['How', 'are', 'you', 'doing'], 'greetings'), (['What', "'s", 'up'], 'greetings'), (['What', 'is', 'the', 'weather'], 'weather'), (['How', 'is', 'it', 'outside'], 'weather'), (['Goodbye'], 'greeting'), (['See', 'you'], 'greeting'), (['Hello'], 'greetings'), (['Hi'], 'greetings'), (['Hey'], 'greetings'), (['How', 'are', 'you', 'doing'], 'greetings'), (['What', "'s", 'up'], 'greetings'), (['What', 'is', 'the', 'weather'], 'weather'), (['How', 'is', 'it', 'outside'], 'weather'), (['Goodbye'], 'greeting'), (['See', 'you'], 'greeting')]


In [26]:
words = []
classes = []
documents = []
ignore_letters = [".",",","?","!"]

In [27]:
for con in conversation["conversation"]:
    for pat in con["patterns"]:
        word_list = nltk.word_tokenize(pat)
        words.extend(word_list)
        documents.append((word_list, con["tag"]))
        if con['tag'] not in classes:
            classes.append(con['tag'])

In [28]:
print(documents)

[(['Hello'], 'greetings'), (['Hi'], 'greetings'), (['Hey'], 'greetings'), (['How', 'are', 'you', 'doing'], 'greetings'), (['What', "'s", 'up'], 'greetings'), (['What', 'is', 'the', 'weather'], 'weather'), (['How', 'is', 'it', 'outside'], 'weather'), (['Goodbye'], 'greeting'), (['See', 'you'], 'greeting')]


In [29]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)

["'s", 'Goodbye', 'Hello', 'Hey', 'Hi', 'How', 'See', 'What', 'are', 'doing', 'is', 'it', 'outside', 'the', 'up', 'weather', 'you']


In [30]:
classes = sorted(set(classes))

In [46]:
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(classes, open("classes.pkl","wb"))

In [32]:
training = []
output_empty = [0] * len(classes)

In [34]:
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [35]:
random.shuffle(training)
training = np.array(training)

C:\Users\petrd\AppData\Local\Temp/ipykernel_21008/2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [36]:
xtrain = list(training[:, 0])
ytrain = list(training[:, 1])

In [37]:
model = Sequential()
model.add(Dense(128, input_shape=(len(xtrain[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(ytrain[0]), activation="softmax"))

In [44]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])


C:\Users\petrd\anaconda3\envs\r_env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [47]:
hist = model.fit(np.array(xtrain), np.array(ytrain), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("Successfully done!")

Epoch 1/200
2/2 [==============================] - 0s 6ms/step - loss: 0.3915 - accuracy: 0.8889
Epoch 2/200
2/2 [==============================] - 0s 5ms/step - loss: 0.2363 - accuracy: 0.8889
Epoch 3/200
2/2 [==============================] - 0s 7ms/step - loss: 0.3893 - accuracy: 0.8889
Epoch 4/200
2/2 [==============================] - 0s 7ms/step - loss: 0.2715 - accuracy: 0.8889
Epoch 5/200
2/2 [==============================] - 0s 5ms/step - loss: 0.2673 - accuracy: 0.8889
Epoch 6/200
2/2 [==============================] - 0s 7ms/step - loss: 0.2713 - accuracy: 0.8889
Epoch 7/200
2/2 [==============================] - 0s 6ms/step - loss: 0.2677 - accuracy: 0.8889
Epoch 8/200
2/2 [==============================] - 0s 5ms/step - loss: 0.3118 - accuracy: 0.8889
Epoch 9/200
2/2 [==============================] - 0s 6ms/step - loss: 0.3039 - accuracy: 0.8889
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 0.3838 - accuracy: 0.8889
Epoch 11/200
2/2 [===========